In [1]:
import numpy as np

import mne
mne.set_log_level(verbose='CRITICAL')
from mne.datasets import multimodal

import os
import glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

import mneflow
print(mneflow.__version__)

0.5.6


In [2]:
root = '/scratch/alr664/multiple_affix'
meg = root + '/meg'
logs = root + '/logs'

full_dataset = ["A0394", "A0421", "A0446", "A0451", "A0468", "A0484", "A0495", "A0502", "A0503", "A0508", 
                "A0509", "A0512", "A0513", "A0514", "A0516", "A0517", "A0518", "A0519", "A0520", "A0521", 
                "A0522", "A0523", "A0524", "A0525"]

In [4]:
subjects = [subj for subj in os.listdir(meg) if not subj.startswith('.')]
subjects
#len(subjects)

['A0522',
 'A0508',
 'A0518',
 'A0514',
 'A0503',
 'A0484',
 'A0524',
 'A0516',
 'A0512',
 'A0520',
 'A0468',
 'A0495',
 'A0451',
 'A0517',
 'A0513',
 'A0525',
 'A0421',
 'A0502',
 'A0519',
 'A0521',
 'A0446',
 'A0523',
 'A0394',
 'A0509']

In [4]:
epochs_list = []

for subject in full_dataset:
    subj_epoch_path = meg + '/' + subject + '/' + subject + '_rejection-epo.fif'
    print(subj_epoch_path)
    subj_epoch = mne.read_epochs(subj_epoch_path)
    subj_epoch_mag = subj_epoch.pick_types(meg='mag')
    epochs_list.append(subj_epoch)

print("Done!")

epochs = mne.concatenate_epochs(epochs_list, on_mismatch='ignore')

print("Done!.")

/scratch/alr664/multiple_affix/meg/A0394/A0394_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0421/A0421_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0446/A0446_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0451/A0451_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0468/A0468_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0484/A0484_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0495/A0495_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0502/A0502_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0503/A0503_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0508/A0508_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0509/A0509_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0512/A0512_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0513/A0513_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0514/A0514_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0516/A0516_rejection-epo.fif
/scratch/alr664/multiple_

In [ ]:
epochs.event_id
epochs = mne.epochs.combine_event_ids(epochs, ['0Suff NW', '1Suff PseudoStemNW', '1Suff RealStemNW', '2Suff RealStemNW', '2Suff PseudoStemNW'],  {'NW': 100},  True)
epochs = mne.epochs.combine_event_ids(epochs, ['1Suff w/ Lat.', '0Suff w/ Lat','0Suff w/o Lat.', '1Suff w/o Lat.', '2Suff w/ Lat.', '2Suff w/o Lat.', '2Suff Composite'],  {'W': 101},  True)
epochs.event_id

In [5]:
epochs.event_id
epochs = mne.epochs.combine_event_ids(epochs, ['0Suff NW', '0Suff w/o Lat.', '0Suff w/ Lat'],  {'0Suff': 100},  True)
epochs = mne.epochs.combine_event_ids(epochs, ['1Suff PseudoStemNW', '1Suff RealStemNW', '1Suff w/ Lat.', '1Suff w/o Lat.'],  {'1Suff': 101},  True)
epochs = mne.epochs.combine_event_ids(epochs, ['2Suff RealStemNW', '2Suff PseudoStemNW', '2Suff w/ Lat.', '2Suff w/o Lat.', '2Suff Composite'],  {'2Suff': 102},  True)

In [6]:
epochs.event_id

{'0Suff': 100, '1Suff': 101, '2Suff': 102}

In [7]:
from collections import Counter
Counter([event[2] for event in epochs.events])

Counter({102: 19296, 101: 13632, 100: 12336})

In [8]:
epochs.times

array([-0.096, -0.088, -0.08 , -0.072, -0.064, -0.056, -0.048, -0.04 ,
       -0.032, -0.024, -0.016, -0.008,  0.   ,  0.008,  0.016,  0.024,
        0.032,  0.04 ,  0.048,  0.056,  0.064,  0.072,  0.08 ,  0.088,
        0.096,  0.104,  0.112,  0.12 ,  0.128,  0.136,  0.144,  0.152,
        0.16 ,  0.168,  0.176,  0.184,  0.192,  0.2  ,  0.208,  0.216,
        0.224,  0.232,  0.24 ,  0.248,  0.256,  0.264,  0.272,  0.28 ,
        0.288,  0.296,  0.304,  0.312,  0.32 ,  0.328,  0.336,  0.344,
        0.352,  0.36 ,  0.368,  0.376,  0.384,  0.392,  0.4  ,  0.408,
        0.416,  0.424,  0.432,  0.44 ,  0.448,  0.456,  0.464,  0.472,
        0.48 ,  0.488,  0.496,  0.504,  0.512,  0.52 ,  0.528,  0.536,
        0.544,  0.552,  0.56 ,  0.568,  0.576,  0.584,  0.592,  0.6  ,
        0.608,  0.616,  0.624,  0.632,  0.64 ,  0.648,  0.656,  0.664,
        0.672,  0.68 ,  0.688,  0.696,  0.704,  0.712,  0.72 ,  0.728,
        0.736,  0.744,  0.752,  0.76 ,  0.768,  0.776,  0.784,  0.792,
      

In [9]:
path = './data/'
data_id = 'meg_epochs'

import_opt = dict(path=path,
                  data_id=data_id,
                  input_type='trials',
                  target_type='int',
                  n_folds= 5,
                  test_set = 'holdout',
                  overwrite=True,
                  picks={'meg':'grad'},
                  scale=False,
                  crop_baseline=True,
                  decimate=None,
                  scale_interval=(0, 13),
                  )

In [10]:
meta = mneflow.produce_tfrecords(epochs, **import_opt)

processing epochs
Input shapes: X (n, ch, t) :  (45264, 207, 113) y (n, [signal_channels], y_shape) :  (45264, 1) 
 input_type :  trials target_type :  int segment_y :  False
Preprocessing:
n: 45264
Splitting into: 6 folds x 7544
Preprocessed: (45264, 1, 113, 207) (45264, 1) folds: 6 x 7544
Preprocessed targets:  (45264, 1)
Prepocessed sample shape: (1, 113, 207)
Target shape actual/metadata:  (3,) (3,)
Saving TFRecord# 0
Updating: meta.data
Updating: meta.preprocessing


In [11]:
dataset = mneflow.Dataset(meta)

Updating: meta.data


In [12]:
lfcnn_params = dict(n_latent=32,
                  filter_length=7,
                  nonlin = tf.nn.relu,
                  padding = 'SAME',
                  pooling = 2,
                  pool_type='max',
                  dropout = .5,
                  l1_scope = ["weights"],
                  l1=3e-4)

meta.update(model_specs=lfcnn_params)

model = mneflow.models.LFCNN(meta)
model.build()

Updating: meta.model_specs
Updating: meta.data
Setting reg for dmx, to l1
Built: dmx input: (None, 1, 113, 207)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 113, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 57, 32)
Input shape: (1, 113, 207)
y_pred: (None, 3)
Initialization complete!


In [13]:
model.train(n_epochs=20, eval_step=50, mode='cv')

Updating: meta.train_params
Class weights:  None
Running cross-validation with 5 folds
fold: 0
Epoch 1/20
50/50 - 9s - 189ms/step - cat_ACC: 0.4060 - loss: 1.2981 - val_cat_ACC: 0.4199 - val_loss: 1.2737
Epoch 2/20
50/50 - 4s - 80ms/step - cat_ACC: 0.4100 - loss: 1.2699 - val_cat_ACC: 0.4199 - val_loss: 1.2476
Epoch 3/20
50/50 - 3s - 60ms/step - cat_ACC: 0.4360 - loss: 1.2307 - val_cat_ACC: 0.4199 - val_loss: 1.2259
Epoch 4/20
50/50 - 2s - 47ms/step - cat_ACC: 0.4080 - loss: 1.2190 - val_cat_ACC: 0.4199 - val_loss: 1.2004
Epoch 5/20
50/50 - 2s - 47ms/step - cat_ACC: 0.4212 - loss: 1.1912 - val_cat_ACC: 0.4199 - val_loss: 1.1823
Epoch 6/20
50/50 - 3s - 51ms/step - cat_ACC: 0.4372 - loss: 1.1746 - val_cat_ACC: 0.4199 - val_loss: 1.1658
Epoch 7/20
50/50 - 3s - 56ms/step - cat_ACC: 0.4128 - loss: 1.1636 - val_cat_ACC: 0.4199 - val_loss: 1.1531
Epoch 8/20
50/50 - 3s - 65ms/step - cat_ACC: 0.4188 - loss: 1.1525 - val_cat_ACC: 0.4199 - val_loss: 1.1422
Epoch 9/20
50/50 - 3s - 57ms/step - cat_

In [14]:
test_loss, test_acc = model.evaluate(meta.data['test_paths'])
print("Test set: Loss = {:.4f} Accuracy = {:.4f}".format(test_loss, test_acc))

Test set: Loss = 1.0859 Accuracy = 0.4274


In [15]:
model = mneflow.models.VARCNN(meta)
model.build()

Updating: meta.data
Setting reg for dmx, to l1
Built: dmx input: (None, 1, 113, 207)
input_shape: (None, 1, 113, 32)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 113, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 57, 32)
Input shape: (1, 113, 207)
y_pred: (None, 3)
Initialization complete!


In [18]:
model.train(n_epochs=20, eval_step=50, mode='cv')

Updating: meta.train_params
Class weights:  None
Running cross-validation with 5 folds
fold: 0
Epoch 1/20
50/50 - 5s - 93ms/step - cat_ACC: 0.4308 - loss: 1.1778 - val_cat_ACC: 0.4199 - val_loss: 1.1782
Epoch 2/20
50/50 - 3s - 67ms/step - cat_ACC: 0.4184 - loss: 1.1783 - val_cat_ACC: 0.4199 - val_loss: 1.1744
Epoch 3/20
50/50 - 2s - 42ms/step - cat_ACC: 0.4408 - loss: 1.1657 - val_cat_ACC: 0.4199 - val_loss: 1.1730
Epoch 4/20
50/50 - 2s - 42ms/step - cat_ACC: 0.4184 - loss: 1.1711 - val_cat_ACC: 0.4199 - val_loss: 1.1655
Epoch 5/20
50/50 - 2s - 42ms/step - cat_ACC: 0.4240 - loss: 1.1621 - val_cat_ACC: 0.4199 - val_loss: 1.1617
Epoch 6/20
50/50 - 2s - 43ms/step - cat_ACC: 0.4404 - loss: 1.1529 - val_cat_ACC: 0.4199 - val_loss: 1.1576
Epoch 7/20
50/50 - 2s - 42ms/step - cat_ACC: 0.4196 - loss: 1.1568 - val_cat_ACC: 0.4199 - val_loss: 1.1540
Epoch 8/20
50/50 - 2s - 43ms/step - cat_ACC: 0.4248 - loss: 1.1538 - val_cat_ACC: 0.4199 - val_loss: 1.1517
Epoch 9/20
50/50 - 2s - 42ms/step - cat_A

In [19]:
test_loss, test_acc = model.evaluate(meta.data['test_paths'])
print("Test set: Loss = {:.4f} Accuracy = {:.4f}".format(test_loss, test_acc))

Test set: Loss = 1.0849 Accuracy = 0.4274
